Het idee is als volgt: we gaan steeds vragen stellen in triple (max) en daarmee steeds 1/4e uitsluiten.

De mogelijke range wordt bewaard door een lijst van tuples die steeds het begin en eind van een stukje mogelijke getallen representeren, eindpunten inclusief.

Bij elke gok kan het antwoord ook zijn dat dat het is, dus die mogelijkheid sla ik in de uitleg even over... Just check after each guess. 

De interessante cases zijn dus dat het hoger of lager ligt. Of het een leugen is of niet hou ik helemaal in het midden, we gaan gewoon vragen stellen tot we contradicties tegenkomen. Dat gebeurd het makkelijkst als er G op een hoger getal wordt geantwoord en L op een lager getal, dus dat gaan we opzoeken! Dan doordat er geen twee leugens achter elkaar mogen volgen, stellen we de tweede vraag nog eens, en kunnen een range uitsluiten!

Een voorbeeld:

`N = 11`, dus de mogelijke start range is (1,11)

1. we vragen dan eerst heel neutraal in het midden: 6
1. Stel ze zegt `L`. Om de contradictie op te zoeken vragen we juist _hoger_: 9 (midden van (7,11))
  1. Als nu weer `L` zegt, dan weten we dat één van de twee waar is, dus het is minstens lager dan 9. We kunnen dan (9,11) uitsluiten (9 ook omdat ze anders wel `E` had gezegd). __We hoeven dus ook niet nog eens 9 te vragen, maar beginnen opnieuw met de mogelijke range (1,8).__
  1. Als ze `G` zegt, dan hebben we de tegenstelling! 
1. Koppig vragen we nog eens 9
  1. Zegt ze nog eens `G`, dan was dat geen leugen, dus kunnen we het hele stuk onder en inclusief 9 uitsluiten. __We kunnen nu verder met de nieuwe range (10,11).__
  1. Zegt ze nu opeens `L`, dan weten we het minst: we weten dat het 
    - óf de `G` voor de negen een leugen was, en het dus onder de 6 én onder de 9 ligt (dus onder de 6), 
    - óf de `L` voor de negen en het dus boven de 9 ligt. 
    - We weten dus vooral wel dat het _niet_ in de range (6,9) ligt. __We kunnen dus verder met de nieuwe range [(1,5),(10,11)]__

En repeat!

In [40]:
from typing import List, Tuple

def get_next_guess_from_middle(still_possible: List[Tuple]) -> int:
    """
    Gets the middle value of a list of ranges, provided as tuples (start, end, both inclusive)
    """
    still_possible = sorted(still_possible)
    int_lens = [x2 - x1 +1 for (x1, x2) in still_possible]

    options_left = sum(int_lens)
    mid_point = int(options_left/2)
    current_point = 0
    next_guess = 1
    for ((x1,x2), l) in zip(still_possible, int_lens):
        if current_point + l <= mid_point:
            current_point += l
        else:
            next_guess = int(x1 + (mid_point - current_point))
            break
    return next_guess

def get_second_guess(still_possible: List[Tuple], first_guess: int, answer_higher_than: bool) -> int:
    """
    If you have an answer to the first guess, use this to choose the mid point of the range ON THE OPPOSITE SIDE!
    """
    if answer_higher_than:
        # Exclude all the values over the first guess to create a contraction
        search_range = get_new_range(still_possible,(first_guess,max([x2 for x1,x2 in still_possible]) + 10))
    else:
        # vice versa
        search_range = get_new_range(still_possible, (1, first_guess))
    return get_next_guess_from_middle(search_range)

def get_new_range(old_range: List[Tuple], excluded_range: Tuple):
    """
    With old_range being a list of tuples, indicating a range (start, end, both inclusive),
    the excluded_range is detracted from all provided intervals.
    Both endpoints of the excluded range are also removed.
    """
    new_range = []
    y1,y2 = excluded_range
    for x1,x2 in old_range:
        #no overlap
        if x1 > y2 or x2 < y1:
            new_range.append((x1,x2))
        #with overlap
        else:
            #exclusion before start of this range
            if y1 > x1:
                new_range.append((x1, y1-1))
            #exclusion beyond end of this range
            if y2 < x2:
                new_range.append((y2+1, x2))
    return new_range
            

def make_a_guess(guess: int) -> Tuple[bool, bool]:
    """
    Return if you found the number, and if the value is higher than the guess
    (according to this lier, that is...)
    """
    print(guess)
    answer = input()
    if answer == 'E':
        return True, None
    elif answer == 'G':
        return False, True
    else:
        return False, False
    
    
def run(verbose = False):

    max_int = int(input())

    new_range = [(1,max_int)]
    
    guess_point = 0

    while True:
        if verbose:
            print(new_range)
            
        if guess_point == 0:
            first_guess = get_next_guess_from_middle(new_range)
            found_it, first_answer_higher_than = make_a_guess(first_guess)
            if found_it:
                break
            guess_point += 1
            
        elif guess_point == 1:
            second_guess = get_second_guess(new_range, first_guess, first_answer_higher_than)
            found_it, second_answer_higher_than = make_a_guess(second_guess)
            if found_it:
                break
            if first_answer_higher_than != second_answer_higher_than:
                guess_point += 1
            else:
                if first_answer_higher_than:
                    new_range = get_new_range(new_range, (1,second_guess))
                else:
                    new_range = get_new_range(new_range, (second_guess, max_int))
                guess_point = 0
                
        elif guess_point == 2:
            found_it, third_answer_higher_than = make_a_guess(second_guess)
            if found_it:
                break
            if second_answer_higher_than != third_answer_higher_than:
                new_range = get_new_range(new_range,
                                          (min(first_guess, second_guess),
                                           max(first_guess, second_guess)))                      
            else:
                if second_answer_higher_than:
                    new_range = get_new_range(new_range, (1,second_guess))
                else:
                    new_range = get_new_range(new_range, (second_guess, max_int))
            guess_point = 0

        if found_it:
            break

# run()

## Voorbeeld IRL

Voor het voorbeeld van boven, optie 2A, probeer:
- 11
- L
- L
- E

In [41]:
run(True)

11
[(1, 11)]
6
L
[(1, 11)]
9
L
[(1, 8)]
5
E


Voor het voorbeeld van boven, optie 3A, probeer:
- 11
- L
- G
- G
- E

In [42]:
run(True)

11
[(1, 11)]
6
L
[(1, 11)]
9
G
[(1, 11)]
9
G
[(10, 11)]
11
E


Voor het voorbeeld van boven, optie 3B, probeer:
- 11
- L
- G
- L
- E

In [43]:
run(True)

11
[(1, 11)]
6
L
[(1, 11)]
9
G
[(1, 11)]
9
L
[(1, 5), (10, 11)]
4
E


## Voorbeelden midden van ranges

In [44]:
get_next_guess_from_middle([(1,11)])

6

1-10 heeft niet echt een midden: het maakt niet uit of je 5 of 6 kiest, je houdt altijd 4 over aan één kant...

In [45]:
get_next_guess_from_middle([(1,10)])

6

In [46]:
get_next_guess_from_middle([(1,1)])

1

[1,3] aan de ene kant, [5,6] aan de andere:

In [48]:
get_next_guess_from_middle([(1,1),(3,6)])

4

In [49]:
get_next_guess_from_middle([(1,1),(3,3),(5,5)])

3

In [51]:
get_next_guess_from_middle([(1,1),(2,2),(3,3),(5,5)])

3

## Voorbeelden van range exclusion

In [52]:
get_new_range([(1,20)],(5,15))

[(1, 4), (16, 20)]

In [53]:
get_new_range([(1, 4), (16, 20)],(4,10))

[(1, 3), (16, 20)]

In [54]:
get_new_range([(1, 4), (16, 20)], (10,25))

[(1, 4)]

Een volgende voorbeeld zou nooit uit dit algoritme kunnen komen: je snijd alleen wegm dus er zullen geen overlappende ranges over blijven, maar toch:

In [55]:
get_new_range([(1,20),(10,15),(5,17),(1,11)], (8,11))

[(1, 7), (12, 20), (12, 15), (5, 7), (12, 17), (1, 7)]